# Single Excitation Configuration Interaction

In this notebook, we will be studying the effect of single excitation configuration interaction on the resuls obtained from constrained unrestricted hartree-fock. We will use the package psi4 for reference data and some basic methods. The UHF and CUHF code will come from the compChem package. We will begin this discussion with a theoretical approach to CIS and on the implementation in a normal UHF system. We will then proceed to extrapolate these findings to CUHF. The theory will be developed for RHF and then implemented for UHF, with the nessecary adjustments.